In [4]:
from os import path, listdir

import matplotlib as plt
import pandas as pd

from process_results import recursive_file_search

RESULTS_DIR = path.abspath("../results")

In [27]:
# Find all results
result_filter = lambda x: "processed-" in x

all_results = list(recursive_file_search(RESULTS_DIR, result_filter))
result_prefix = path.commonprefix(c5n_results)

# Find all c5n directories
c5n_filter = lambda x: "c5n" in x and "predictions-" not in x
c5n_results = list(filter(c5n_filter, all_results))



c5n_results


['/home/niels/fc-microbenchmark/results/RRc5n/002/processed-results-poisson-25000-1hr-mem.txt',
 '/home/niels/fc-microbenchmark/results/RRc5n/001/processed-results-poisson-50000-1hr-cpu.txt',
 '/home/niels/fc-microbenchmark/results/RRc5n/003/processed-results-poisson-12500-1hr-mem.txt',
 '/home/niels/fc-microbenchmark/results/RRc5n/004/processed-results-poisson-25000-1hr-75cpu25mem.txt',
 '/home/niels/fc-microbenchmark/results/CFSc5n/002/processed-results-poisson-25000-1hr-mem.txt',
 '/home/niels/fc-microbenchmark/results/CFSc5n/001/processed-results-poisson-50000-1hr-mem.txt',
 '/home/niels/fc-microbenchmark/results/CFSc5n/001/processed-results-poisson-50000-1hr-cpu.txt',
 '/home/niels/fc-microbenchmark/results/CFSc5n/003/processed-results-poisson-12500-1hr-mem.txt',
 '/home/niels/fc-microbenchmark/results/CFSc5n/004/processed-results-poisson-25000-1hr-75cpu25mem.txt',
 '/home/niels/fc-microbenchmark/results/BATCHc5n/002/processed-results-poisson-12500-1hr-mem.txt',
 '/home/niels/fc-m